In [2]:
import argparse
import gzip
import json
import os
import time
import sys

from sqlalchemy import create_engine, text

In [3]:
OASIS_DB_DIR = '../../work/'
OASIS_DB_NAME = 'OASis_9mers_v1.db'
OASIS_DB_PATH = OASIS_DB_DIR + '/' + OASIS_DB_NAME

In [4]:
OUTPUT_OASIS_9MERS_PATH = '../../promb/resources/OASis_9mers_v1_10perc_subjects.txt.gz'

In [19]:
MIN_PERCENT_SUBJECTS = 10

In [13]:
# Download database file
#!wget -O {OASIS_DB_DIR}/{OASIS_DB_NAME}.gz https://zenodo.org/record/5164685/files/OASis_9mers_v1.db.gz
# Unzip
#cd {OASIS_DB_DIR}; gunzip {OASIS_DB_NAME}.gz

In [14]:
engine = create_engine('sqlite:///' + os.path.abspath(OASIS_DB_PATH))

In [20]:
with engine.connect() as conn:
    heavy_subject_ids = [int(row[0]) for row in conn.execute(
        text("SELECT id FROM subjects WHERE CompleteHeavySeqs >= 10000 AND subjects.StudyPath <> 'Corcoran_2016'")
    ).fetchall()]
    
    light_subject_ids = [int(row[0]) for row in conn.execute(
        text("SELECT id FROM subjects WHERE CompleteLightSeqs >= 10000 AND subjects.StudyPath <> 'Corcoran_2016'")
    ).fetchall()]
    
    min_heavy_subjects = len(heavy_subject_ids) * MIN_PERCENT_SUBJECTS / 100
    min_light_subjects = len(light_subject_ids) * MIN_PERCENT_SUBJECTS / 100

print('Min heavy subjects: ', min_heavy_subjects)
print('Min light subjects: ', min_light_subjects)

Min heavy subjects:  22.4
Min light subjects:  15.4


In [22]:
min_subjects = min(min_heavy_subjects, min_light_subjects)
min_subjects

15.4

In [21]:
subject_ids = sorted(set(heavy_subject_ids) | set(light_subject_ids))
len(subject_ids)

230

In [23]:
%%time

with engine.connect() as conn:
    print('Loading peptides...')
    peptides = [row[0] for row in conn.execute(text(
        f'SELECT peptide, COUNT(peptide) as num_subjects '
        f'FROM peptides WHERE peptides.subject IN ({",".join(map(str, subject_ids))}) '
        f'GROUP BY peptide HAVING num_subjects > {min_subjects}'
    )).fetchall()]
    print(f'Loaded {len(peptides):,} peptides')

Loading peptides...
Loaded 5,725,303 peptides
CPU times: user 3min 40s, sys: 46 s, total: 4min 26s
Wall time: 5min 30s


In [27]:
print(int(sys.getsizeof(peptides)/1024/1024), 'MB')

47 MB


In [28]:
with gzip.open(OUTPUT_OASIS_9MERS_PATH, 'wt') as f:
    for peptide in sorted(peptides):
        f.write(peptide)
        f.write('\n')

In [5]:
!ls -lh {OUTPUT_OASIS_9MERS_PATH}

-rw-r--r--  1 prihodad  staff    13M Feb 14 13:58 ../../promb/resources/OASis_9mers_v1_10perc_subjects.txt.gz


In [6]:
!gunzip -c {OUTPUT_OASIS_9MERS_PATH} | head

AAAAAAVYY
AAAAADAFD
AAAAADTAV
AAAAADYWG
AAAAAFDIW
AAAAAFDYW
AAAAAIYYC
AAAAALDYW
AAAAALYYC
AAAAAVDYC
gunzip: error writing to output: Broken pipe
gunzip: ../../promb/resources/OASis_9mers_v1_10perc_subjects.txt.gz: uncompress failed
